In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization
import numpy as np


2025-07-11 07:46:25.013233: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752219985.225224      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752219985.294375      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
sentences = [
    "I love this movie",
    "This is a great product",
    "I feel awesome",
    "It’s a fantastic day",
    "I hate this",
    "This is terrible",
    "I feel bad",
    "It’s a horrible experience"
]

labels = [1, 1, 1, 1, 0, 0, 0, 0]  # 1 = Positive, 0 = Negative


In [3]:
vectorizer = TextVectorization(max_tokens=1000, output_sequence_length=5)
vectorizer.adapt(sentences)


I0000 00:00:1752219999.318518      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [4]:
X = vectorizer(sentences)
y = np.array(labels)


In [5]:
model = Sequential([
    Embedding(input_dim=1000, output_dim=10),
    GlobalAveragePooling1D(),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [6]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [7]:
model.fit(X, y, epochs=30, verbose=0)


I0000 00:00:1752220002.271764      57 service.cc:148] XLA service 0x7dd738022af0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752220002.272533      57 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1752220002.496865      57 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1752220003.030885      57 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [8]:
test_sentences = ["I am so happy", "This is the worst", "I love this weather"]
test_vectorized = vectorizer(test_sentences)


In [9]:
predictions = model.predict(test_vectorized)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step


In [10]:
for sentence, pred in zip(test_sentences, predictions):
    label = "Positive" if pred > 0.5 else "Negative"
    print(f"'{sentence}' → {label} (Confidence: {pred[0]:.2f})")


'I am so happy' → Negative (Confidence: 0.50)
'This is the worst' → Negative (Confidence: 0.50)
'I love this weather' → Positive (Confidence: 0.51)
